# Higher order functions

Functions allow to perform certain compound operations without considering a specific input. They represent the idea of how to do those operations. There are certain patterns that are generated by the interaction of functions. 

## Function as arguments

If a set of functions share the same suite structure but differ in a single task, the general procedure underlying those suites can be made a more general function that takes the same arguments plus an extra argument specifying the distinct task. That task can be expressed as another function. Thus the more general function has a function as input. We see a new kind of abstraction: some functions express general methods of computation, independent of the particular functions they call. The procedure model for evaluation of call functions still applies when functions are used as arguments.

As an example of a general function, consider a type of function whose task is to improve some numerical value iteratively. It needs a procedure to improve an initial guess (`update`) and a procedure to stop the loop (`close`).

```python
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess
```

## Nested definitions

Problems with function as arguments:
1. If the general function is to be used several times, the programs becomes cluttered with names of small functions.
2. Fixed number of arguments on specific and general functions.

A solution for this problems are **nested function definitions**, which consist on defining functions inside functions. Like local assignment, local `def` statements only affect the current local frame. These functions are only in scope while its superior function is being evaluated.

Locally defined functions also have access to the name bindings in the scope in which they are defined. This discipline of sharing names among nested definitions is called **lexical scoping**. We require two extensions to our environment model to enable lexical scoping.
1. Each user-defined function has a **parent environment**: the environment in which it was defined. When a function is defined in the global environment (outside of a function), its parent environment is the global environment.
2. When a user-defined function is called, its local frame **extends** its parent environment. The environments thus expand the chain of frames they contain, always ending in the global frame. A value of an expression can be resolved by looking at these frames (from the first frame to the global frame). A local function can access the environment of the enclosing function, because the body of the local function is evaluated in an environment that extends the evaluation environment in which it was defined. Locally defined functions are often called **closures**.

Now function values have a **parent**. The parent of a function value is the first frame of the environment in which that function was defined. When a user-defined function is called, the frame created has the same parent as that function.

## Functions as Returned Values

We can have functions whose returned values are themselves functions. An important feature of lexically scoped programming languages is that locally defined functions maintain their parent environment when they are returned, because it is where they were first defined. An example of functions as returned values is the composition of two functions, `f` and `g`, to return a composed function `h = compose(f,g)`.

With this type of functions we can have functions with _two calls_. For instance, in `h = compose(f,g)`, if we want to evaluate `h(5)` it is equivalent to `compose(f,g)(5)`, because first it is evaluated the operand, `compose(f,g)`, which is an expression that evaluates to a function. 

## Currying

The process of transforming a multiple-argument function into a chain of single-argument functions is called **currying**. It is used in `map` patterns (such as `linspace`) or programming languages with only single-argument functions (Haskell). 

Currying and uncurrying can be systematically done for any two functions as follows 
```python
def curry2(f):
    """Return a curried version of the given two-argument function."""
    def g(x):
        def h(y):
            return f(x, y)
        return h
    return g

def uncurry2(g):
    """Return a two-argument version of the given curried function."""
    def f(x, y):
        return g(x)(y)
    return f
```
This transforms `f(x,y)` into `g(x)(y)` and viceversa.

## Lambda functions

Lambda expressions evaluate to unnamed functions. They are functions created on the fly. Syntax: `lambda x: f(x)`, which means _A function that takes x and returns f(x)_. There are no statements in lambda expressions. The only difference to `def` statements is that lambda expressions do not bind a name to the body of the function, something which is done automatically in `def` statements (intrinsic names). Lambda expressions get a name only when bounded to a name. Lambda expressions are a type of anonymous functions. As in `def` statements the return is not evaluated until the lambda is called.

As lambda expressions evaluate to functions they can be used as an operator on a call expression: `(lambda x: x)(5)`.

## Abstractions and First-Class Functions

In Python, functions are first-class elements, which are caracterized by:
1. They may be bound to names.
2. They may be passed as arguments to functions.
3. They may be returned as the results of functions.
4. They may be included in data structures.

These properties allow to use functions in a very abstract way.

## Function decorators

_Bibliography_ : https://www.thecodeship.com/patterns/guide-to-python-function-decorators/

**Decorators** are wrappers that allow to alter the behavour of a function without actually modifying it.

Syntax:
```python
def my_decorator(func):
    def improved_funct(arg):
        # Do stuff with func
    return improved_funct

@my_decorator
def my_funct(arg):
    # Do stuff
    
my_funct(arg) # Decorated function with augmented behavour.
```
It is equivalent to
```python
def my_decorator(func):
    def improved_funct(arg):
        # Do stuff with func
    return improved_funct

def my_funct(arg):
    # Do stuff
    
my_funct = my_decorator(my_funct) # Bind my_funct to my_decorator(my_funct)
my_funct(arg)
```
The first form is a kind of _syntactic sugar_ (syntax of a programming language designed to make programs easier to read, but which do not add any functionality). To add more than one decorator put in separate lines before the function `@decorator1`, `@decorator2`... which means `decorator1(decorator2(...))`.

## HOFs at Hog project

**Commentary function structure**: a parent function with some input variables that returns another function with some arguments. This returned function has the input variables in scope. By calling the returned function with the arguments it can update the value of the input variables of the parent function.

Syntax:
```python
def parent_func(my_var):
    def func(arg1, arg2):
#         Do stuff.
#         Define my_new_var
        return parent_func(my_new_var)
    return func
```

# Recursive Functions

A function is called **recursive** if the body of the function calls the function itself, either directly or indirectly. 

## The Anatomy of Recursive Functions

Recursive functions share a common structure:
- Base case: a conditional statement that defines the behavior of the function for the inputs that are simplest to process. There might be multiple base cases. Base cases are evaluated without recursive calls. Also note that the base case might not do anything at all.
- Recursive calls: conditional statements for complex cases. They simplify the problem trying to reach the base(s) case(s). In this statements the function calls itself with different arguments trusting that it correctly computes that case (*recursive leap of faith*), i.e., it is treated as a function abstraction. If it computes correctly the result for the present case by induction we hope it will do for the rest. Note that different frames keep track of the different arguments in each call.

The order in which the base case and recursive calls appear is not rigid in the program. The base case might be after the recursive call and there might be some statemets before and after the recursive call. The statements after the recursive call are not evaluated until all the recursion is done. 

## More on recursion

**Iterative functions vs recursive functions**: iterative functions solve the problem by starting from base cases, whereas recursive functions simplify the complex, original problem. Iterative functions must introduce variables to maintain some local state that changes throughout the course of computation. These variables are not present in recursive functions, instead the state of the computation is entirely contained within the structure of the environment and conditional statements using arguments. In recursive functions binding names to values is done in the call expression in a new frame, thus avoiding the nuisance of correctly assigning local names during iteration

**Mutual recursion**: when a recursive procedure is divided among two functions that call each other, the functions are said to be mutually recursive. Mutually recursive functions can be turned into a single recursive function by breaking the abstraction boundary between the two functions. Mutual recursion provides a mechanism for maintaining abstraction within a complicated recursive program.

**Tree recursion**: a type of recursion in which a function calls itself more than once. In each call to the function it branches into multiple smaller calls, like a tree.